In [1]:
import os
import random

import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import tensorflow as tf
from evaluation.environment import TrainingEnvironment, TestingEnvironment

2024-01-12 10:44:42.074480: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-12 10:44:42.074536: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-12 10:44:42.076153: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-12 10:44:42.086033: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-12 10:44:43.835477: W tensorflow/compiler/tf2

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
gpus = tf.config.experimental.list_physical_devices("GPU")
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        # Select GPU number 1
        tf.config.experimental.set_visible_devices(gpus[0], "GPU")
        logical_gpus = tf.config.experimental.list_logical_devices("GPU")
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPUs


2024-01-12 10:44:48.338002: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 10394 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:09:00.0, compute capability: 6.1


In [3]:
# Official hyperparameters for this competition (do not modify)
N_TRAIN_USERS = 1000
N_TEST_USERS = 2000
N_ITEMS = 209527
HORIZON = 2000
TEST_EPISODES = 5
SLATE_SIZE = 5

In [4]:
LEARNING_RATE = 2e-5
EMBEDDING_SIZE = 512
N_EPOCHS = 2500
TRAIN_EPISODES = 50
COLLABRATIVE_SLATE_SIZE = 5
CONTENT_SLATE_SIZE = 0

In [5]:
# Dataset paths
USER_DATA = os.path.join("../dataset", "user_data.json")
ITEM_DATA = os.path.join("../dataset", "item_data.json")

# Output file path
OUTPUT_PATH = os.path.join("output", "output_main.csv")

In [6]:
df_user = pd.read_json(USER_DATA, lines=True)
df_user

,user_id,history
0,0,"[42558, 65272, 13353]"
1,1,"[146057, 195688, 143652]"
2,2,"[67551, 85247, 33714]"
3,3,"[116097, 192703, 103229]"
4,4,"[68756, 140123, 135289]"
...,...,...
1995,1995,"[95090, 131393, 130239]"
1996,1996,"[2360, 147130, 8145]"
1997,1997,"[99794, 138694, 157888]"
1998,1998,"[55561, 60372, 51442]"


In [7]:
df_item = pd.read_json(ITEM_DATA, lines=True)
df_item.head()
# df_item["headline"].iloc[df_user.at[0, "history"][0]]

,item_id,headline,short_description
0,0,Over 4 Million Americans Roll Up Sleeves For O...,Health experts said it is too early to predict...
1,1,"American Airlines Flyer Charged, Banned For Li...",He was subdued by passengers and crew when he ...
2,2,23 Of The Funniest Tweets About Cats And Dogs ...,"""Until you have a dog you don't understand wha..."
3,3,The Funniest Tweets From Parents This Week (Se...,"""Accidentally put grown-up toothpaste on my to..."
4,4,Woman Who Called Cops On Black Bird-Watcher Lo...,Amy Cooper accused investment firm Franklin Te...


In [8]:
# Process Data
train_data = []
# res = tf.fill(dims = (2, 3), value = 1)

histories = []
uids = []
ratings = []

for uid in df_user.user_id:
    for his in df_user.at[uid, "history"]:
        uids.append(uid)
        histories.append(his)
        ratings.append(1)
# uids = tf.convert_to_tensor(uids, dtype=tf.float32)
# histories = tf.convert_to_tensor(histories, dtype=tf.float32)
# ratings = tf.convert_to_tensor(ratings, dtype=tf.float32)
# print(type(uids))
# print(df_user.at[uid, "history"])
# df_user.at[uid, "history"].append(55)
# print(df_user.at[uid, "history"])

In [9]:
# df_item["short_description"].values.tolist()

['Health experts said it is too early to predict whether demand would match up with the 171 million doses of the new boosters the U.S. ordered for the fall.',
 "He was subdued by passengers and crew when he fled to the back of the aircraft after the confrontation, according to the U.S. attorney's office in Los Angeles.",
 '"Until you have a dog you don\'t understand what could be eaten."',
 '"Accidentally put grown-up toothpaste on my toddler’s toothbrush and he screamed like I was cleaning his teeth with a Carolina Reaper dipped in Tabasco sauce."',
 'Amy Cooper accused investment firm Franklin Templeton of unfairly firing her and branding her a racist after video of the Central Park encounter went viral.',
 'The 63-year-old woman was seen working at the South Carolina store on Thursday. She was found dead Monday after her family reported her missing, authorities said.',
 '"Who\'s that behind you?" an anchor for New York’s PIX11 asked journalist Michelle Ross as she finished up an int

In [13]:
ds = tf.data.Dataset.from_tensor_slices(df_item["short_description"].values.tolist())

In [19]:
  import tempfile
  path = os.path.join("saved_data")
  # Save a dataset
  # dataset = tf.data.Dataset.range(2)
  # dataset.save(path)
  ds.save(path)
  new_dataset = tf.data.Dataset.load(path)
  # for elem in new_dataset:
  #   print(elem)
  new_dataset.numpy()





AttributeError: '_LoadDataset' object has no attribute 'numpy'

In [ ]:
#! Content-based Model:
#! embedding ===============================================================================
from transformers import CLIPProcessor, CLIPTextModel, AutoTokenizer

model = CLIPTextModel.from_pretrained("openai/clip-vit-base-patch32")
tokenizer = AutoTokenizer.from_pretrained("openai/clip-vit-base-patch32")

# inputs = tokenizer(df_item["short_description"].values.tolist(), padding=True, return_tensors="pt")
inputs = tokenizer(["a photo of a cat", "a photo of a dog"], padding=True, return_tensors="pt")

outputs = model(**inputs)
embedding = outputs.last_hidden_state.detach().numpy()
print(inputs)
# print(embedding)
from sklearn.metrics.pairwise import linear_kernel

# Compute the cosine similarity matrix
cosine_sim = linear_kernel(embedding[0], embedding[1])
print(cosine_sim.shape)
import numpy.linalg as LA

In [ ]:
# handy method for calculating the similarity between 2 word
def cos_sim():
    # id1 = word_to_id[word1]
    # id2 = word_to_id[word2]

    # vec1 = embedding_matrix[id1].numpy()
    # vec2 = embedding_matrix[id2].numpy()

    return np.dot(embedding[0], embedding[1]) / (LA.norm(embedding[0]) * LA.norm(embedding[1]))

# cos_sim()
#! embedding ===============================================================================


# # Compute the cosine similarity matrix
# cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

# # %%
# # Construct a reverse map of indices and movie titles
# indices = pd.Series(df_item_sample.index, index=df_item_sample['headline']).drop_duplicates()

In [ ]:
# Function that takes in movie title as input and outputs most similar movies
# def get_recommendations(idx, cosine_sim=cosine_sim):
#     headline = df_item.at[idx, "headline"]
#     print(f"similar movie for id: {idx} - {headline}")
#     # Get the index of the movie that matches the title
#     # idx = indices[title]

#     # Get the pairwsie similarity scores of all movies with that movie
#     sim_scores = list(enumerate(cosine_sim[idx]))

#     # Sort the movies based on the similarity scores
#     sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

#     # Get the scores of the 10 most similar movies
#     sim_scores = sim_scores[1 : CONTENT_SLATE_SIZE + 1]

#     # Get the movie indices
#     movie_indices = [i[0] for i in sim_scores]

#     # Return the top 10 most similar movies
#     # return df_item['headline'].iloc[movie_indices]

#     return df_item['item_id'].iloc[movie_indices].tolist()

In [ ]:
# content_recs = get_recommendations(165734)
# # print(type(content_recs))
# print(content_recs)

In [ ]:
# Collabrative Model: funk-svd
class FunkSVDRecommender(tf.keras.Model):
    """
    Simplified Funk-SVD recommender model
    """

    def __init__(self, m_users: int, n_items: int, embedding_size: int, learning_rate: float):
        """
        Constructor of the model
        """
        super().__init__()
        self.m = m_users
        self.n = n_items
        self.k = embedding_size
        self.lr = learning_rate

        # user embeddings P
        self.P = tf.Variable(tf.keras.initializers.RandomNormal()(shape=(self.m, self.k)))

        # item embeddings Q
        self.Q = tf.Variable(tf.keras.initializers.RandomNormal()(shape=(self.n, self.k)))

        # optimizer
        self.optimizer = tf.optimizers.Adam(learning_rate=self.lr)
        self.optimizer_update = tf.optimizers.Adam(learning_rate=self.lr)

    @tf.function
    def call(self, user_ids: tf.Tensor, item_ids: tf.Tensor) -> tf.Tensor:
        """
        Forward pass used in training and validating
        """
        # dot product the user and item embeddings corresponding to the observed interaction pairs to produce predictions
        y_pred = tf.reduce_sum(
            tf.gather(self.P, indices=user_ids) * tf.gather(self.Q, indices=item_ids), axis=1
        )

        return y_pred

    @tf.function
    def compute_loss(self, y_true: tf.Tensor, y_pred: tf.Tensor) -> tf.Tensor:
        """
        Compute the MSE loss of the model
        """
        loss = tf.losses.mean_squared_error(y_true, y_pred)

        return loss

    @tf.function
    def train_step(self, uids: tf.Tensor, histories: tf.Tensor, ratings: tf.Tensor) -> tf.Tensor:
        """
        Train the model with one batch
        data: batched user-item interactions
        each record in data is in the format [UserID, MovieID, Rating, Timestamp]
        """
        print("train")
        user_ids = tf.cast(uids, dtype=tf.int32)
        item_ids = tf.cast(histories, dtype=tf.int32)
        y_true = tf.cast(ratings, dtype=tf.float32)

        print(f"uid:{user_ids} items{item_ids} y: {y_true}")

        # compute loss
        with tf.GradientTape() as tape:
            y_pred = self(user_ids, item_ids)
            loss = self.compute_loss(y_true, y_pred)

        # compute gradients
        gradients = tape.gradient(loss, self.trainable_variables)

        # update weights
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))

        return loss

    @tf.function
    def val_step(self, uids: tf.Tensor, histories: tf.Tensor, ratings: tf.Tensor) -> tf.Tensor:
        """
        Validate the model with one batch
        data: batched user-item interactions
        each record in data is in the format [UserID, MovieID, Rating, Timestamp]
        """
        user_ids = tf.cast(uids, dtype=tf.int32)
        item_ids = tf.cast(histories, dtype=tf.int32)
        y_true = tf.cast(ratings, dtype=tf.float32)

        # compute loss
        y_pred = self(user_ids, item_ids)
        loss = self.compute_loss(y_true, y_pred)

        return loss

    @tf.function
    def eval_predict_onestep(self, query: tf.Tensor) -> tf.Tensor:
        """
        Retrieve and return the MovieIDs of the 10 recommended movies given a query
        You should return a tf.Tensor with shape=(10,)
        query will be a tf.Tensor with shape=(2,) and dtype=tf.int64
        query[0] is the UserID of the query
        #### query[1] is the Timestamp of the query
        """
        # dot product the selected user and all item embeddings to produce predictions
        user_id = tf.cast(query, tf.int32)
        y_pred = tf.reduce_sum(tf.gather(self.P, user_id) * self.Q, axis=1)

        # select the top 10 items with highest scores in y_pred
        y_recommends = tf.math.top_k(y_pred, k=COLLABRATIVE_SLATE_SIZE).indices

        return y_recommends

    @tf.function
    def eval_update_onestep(
        self, uids: tf.Tensor, histories: tf.Tensor, ratings: tf.Tensor
    ) -> None:
        # data = data[None, :]  # add a dim on axis 0
        # user_ids = tf.cast(data[:, 0], dtype=tf.int32)
        # item_ids = tf.cast(data[:, 1], dtype=tf.int32)
        # y_true = tf.cast(data[:, 2], dtype=tf.float32)
        user_ids = tf.cast(uids, dtype=tf.int32)
        item_ids = tf.cast(histories, dtype=tf.int32)
        y_true = tf.cast(ratings, dtype=tf.float32)

        # compute loss
        y_pred = self(user_ids, item_ids)
        loss = self.compute_loss(y_true, y_pred)

        # compute loss
        with tf.GradientTape() as tape:
            y_pred = self(user_ids, item_ids)
            loss = self.compute_loss(y_true, y_pred)

        # compute gradients
        gradients = tape.gradient(loss, self.trainable_variables)

        # update weights
        self.optimizer_update.apply_gradients(zip(gradients, self.trainable_variables))

In [ ]:
# selected_slate = [[]] * N_TEST_USERS
# print(len(uids))
# print(len(histories))
# for i in range(len(uids)):
#     u = int(uids[i])
#     print(u)
#     # for hi in range(len(histories)):
#     #     h = histories[hi]
#     #     # print(h)
#     #     selected_slate[u].append(float(h))
#     print(selected_slate[u])
#     selected_slate[u].append(float(histories[i]))
# print("done")
# selected_slate

In [ ]:
model = FunkSVDRecommender(
    m_users=N_TEST_USERS,
    n_items=N_ITEMS,
    embedding_size=EMBEDDING_SIZE,
    learning_rate=LEARNING_RATE,
)
for epoch in range(1, N_EPOCHS + 1):
    train_loss = []
    val_loss = []
    print(f"Epoch {epoch}:")

    # training
    # for data in tqdm(uids, desc='Training'):
    loss = model.train_step(uids, histories, ratings)
    train_loss.append(loss.numpy())

    # # validating
    # for data in tqdm(dataset_val, desc='Validating'):
    loss = model.val_step(uids, histories, ratings)
    val_loss.append(loss.numpy())

    # record losses
    avg_train_loss = np.mean(train_loss)
    avg_val_loss = np.mean(val_loss)
    # train_losses.append(avg_train_loss)
    # val_losses.append(avg_val_loss)

    # print losses
    print(f"Epoch {epoch} train_loss: {avg_train_loss:.4f}, val_loss: {avg_val_loss:.4f}\n")
model.save("model_funk_svd")

In [ ]:
# Initialize the training environment
train_env = TrainingEnvironment()

# selected_slate = [[]] * N_TEST_USERS
# for i in range(uids):
#     u = uids[i]
#     selected_slate[u].append(histories[i])

for _ in range(TRAIN_EPISODES):
    # Reset the training environment (this can be useful when you have finished one episode of simulation and do not want to re-initialize a new environment)
    train_env.reset()

    # Check if there exist any active users in the environment
    # env_has_next_state = train_env.has_next_state()
    while train_env.has_next_state():
        # print(f'There is {"still some" if env_has_next_state else "no"} active users in the training environment.')

        # Get the current user ID
        user_id = train_env.get_state()
        print(f'The current user is user {user_id}.')

        # Get the response of recommending the slate to the current user
        slate = model.eval_predict_onestep([user_id])
        # user_first_his = df_user.at[user_id, 'history'][0]
        # content_slate = get_recommendations(user_first_his)
        # print(type(content_slate))

        # slate = content_slate.append(collab_slate)

        clicked_id, in_environment = train_env.get_response(slate)
        uids_ = []
        histories_ = []
        ratings_ = []
        for his in df_user.at[user_id, "history"]:
            uids_.append(user_id)
            histories_.append(his)
            ratings_.append(1)
        for s in slate:
            uids_.append(user_id)
            histories_.append(s)
            if clicked_id == s:
                ratings_.append(1)
                df_user.at[uid, "history"].append(s)
            else:
                ratings_.append(-1)
        model.eval_update_onestep(uids_, histories_, ratings_)
        if clicked_id != -1:
            print(
                f'The click result of recommending {slate} to user {user_id} is {f"item {clicked_id}" if clicked_id != -1 else f"{clicked_id} (no click)"}.'
            )
            print(
                f'User {user_id} {"is still in" if in_environment else "leaves"} the environment.'
            )
            print(ratings_)

model.save("model_funk_svd")

# Get the normalized session length score of all users
train_score = train_env.get_score()
df_train_score = pd.DataFrame(
    [[user_id, score] for user_id, score in enumerate(train_score)],
    columns=["user_id", "avg_score"],
)
df_train_score

In [ ]:
# Initialize the testing environment
test_env = TestingEnvironment()
scores = []

# The item_ids here is for the random recommender
# item_ids = [i for i in range(N_ITEMS)]

# Repeat the testing process for 5 times
for _ in range(TEST_EPISODES):
    # [TODO] Load your model weights here (in the beginning of each testing episode)
    # [TODO] Code for loading your model weights...
    # model.save('model_funk_svd_update')
    # model = tf.keras.models.load_model('model_funk_svd', compile=False)

    # Start the testing process
    with tqdm(desc="Testing") as pbar:
        # Run as long as there exist some active users
        while test_env.has_next_state():
            # Get the current user id
            cur_user = test_env.get_state()

            # [TODO] Employ your recommendation policy to generate a slate of 5 distinct items
            # [TODO] Code for generating the recommended slate...
            # Here we provide a simple random implementation
            # slate = random.sample(item_ids, k=SLATE_SIZE)
            # model = tf.keras.models.load_model('model_funk_svd_update')
            # print(cur_user)
            collab_slate = model.eval_predict_onestep([cur_user])
            # content_slate = get_recommendations(df_user[0])

            # Get the response of the slate from the environment
            clicked_id, in_environment = test_env.get_response(slate)

            # [TODO] Update your model here (optional)
            # [TODO] You can update your model at each step, or perform a batched update after some interval
            # [TODO] Code for updating your model...

            uids_ = []
            histories_ = []
            ratings_ = []
            for his in df_user.at[cur_user, "history"]:
                uids_.append(cur_user)
                histories_.append(his)
                ratings_.append(1)
            for s in slate:
                uids_.append(cur_user)
                histories_.append(s)
                if clicked_id == s:
                    ratings_.append(1)
                    df_user.at[uid, "history"].append(s)
                else:
                    ratings_.append(-1)

            model.eval_update_onestep(uids_, histories_, ratings_)
            # model.save('model_funk_svd_update')

            # Update the progress indicator
            pbar.update(1)

    # Record the score of this testing episode
    scores.append(test_env.get_score())

    # Reset the testing environment
    test_env.reset()

    # [TODO] Delete or reset your model weights here (in the end of each testing episode)
    # [TODO] Code for deleting your model weights...

In [ ]:
# Calculate the average scores
avg_scores = [np.average(score) for score in zip(*scores)]

# Generate a DataFrame to output the result in a .csv file
df_result = pd.DataFrame(
    [[user_id, avg_score] for user_id, avg_score in enumerate(avg_scores)],
    columns=["user_id", "avg_score"],
)
df_result.to_csv(OUTPUT_PATH, index=False)
df_result